In [10]:
import os
import cv2
import numpy as np
from keras import Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Conv2D, BatchNormalization, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder





In [7]:
# Ruta relativa a la carpeta de imágenes (si el código y las imágenes están en el mismo directorio)
images_path = '/Users/giovannyhidalgo/Desktop/archive/images/images/'

# Función para cargar y redimensionar imágenes
def load_and_resize_image(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        resized_image = cv2.resize(image, (28, 28))
        return resized_image
    else:
        return None

X_data = []
Y_data = []





In [8]:
print(images_path)


/Users/giovannyhidalgo/Desktop/archive/images/images/


In [13]:
# Cargar imágenes y generar datos de entrenamiento
for filename in os.listdir(images_path):
    image_path = os.path.join(images_path, filename)
    image = load_and_resize_image(image_path)
    if image is not None:
        X_data.append(image)
        img_name = filename.split('.')[0]
        Y_data.append(img_name)

        # Generar 100 imágenes adicionales para el Pokémon actual usando el generador de datos
        for _ in range(100):
            random_img = datagen.random_transform(image)
            X_data.append(random_img)
            Y_data.append(img_name)

if not X_data or not Y_data:
    raise ValueError("No se encontraron imágenes válidas en el directorio.")

X_data = np.array(X_data).astype('float32') / 255.0

encoder = LabelEncoder()
Y_data = encoder.fit_transform(Y_data)
Y_data = to_categorical(Y_data)

X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)


In [24]:
np.save('encoder_classes.npy', encoder.classes_)

In [12]:
# Generador de datos de imágenes con configuración optimizada

datagen = ImageDataGenerator(
    rotation_range=45,
    zoom_range=0.30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest')


In [14]:
# Arquitectura del modelo (más profunda para mejorar el rendimiento)
model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# Agregar más capas convolucionales y de normalización aquí
# ...
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Entrenar el modelo con configuración adicional
model.fit(datagen.flow(X_train, Y_train, batch_size=192),
          epochs=20,
          steps_per_epoch=X_train.shape[0] // 192,
          validation_data=(X_test, Y_test),
          callbacks=[EarlyStopping(monitor='loss', min_delta=1e-10, patience=20, verbose=1),
                     ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, verbose=1),
                     ModelCheckpoint(filepath='weights.h5', monitor='loss',
                                     save_best_only=True, verbose=1)])


Epoch 1/20
340/340 [==============================] - ETA: 0s - loss: 5.3999 - accuracy: 0.0589
Epoch 1: loss improved from inf to 5.39990, saving model to weights.h5


/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


340/340 [==============================] - 233s 682ms/step - loss: 5.3999 - accuracy: 0.0589 - val_loss: 4.0840 - val_accuracy: 0.1811 - lr: 0.0010
Epoch 2/20
340/340 [==============================] - ETA: 0s - loss: 3.9364 - accuracy: 0.1947
Epoch 2: loss improved from 5.39990 to 3.93639, saving model to weights.h5
340/340 [==============================] - 309s 907ms/step - loss: 3.9364 - accuracy: 0.1947 - val_loss: 2.9204 - val_accuracy: 0.3609 - lr: 0.0010
Epoch 3/20
340/340 [==============================] - ETA: 0s - loss: 3.1065 - accuracy: 0.3218
Epoch 3: loss improved from 3.93639 to 3.10646, saving model to weights.h5
340/340 [==============================] - 316s 928ms/step - loss: 3.1065 - accuracy: 0.3218 - val_loss: 2.1539 - val_accuracy: 0.5176 - lr: 0.0010
Epoch 4/20
340/340 [==============================] - ETA: 0s - loss: 2.5195 - accuracy: 0.4257
Epoch 4: loss improved from 3.10646 to 2.51948, saving model to weights.h5
340/340 [==============================] - 

In [15]:
# Guardar el modelo
model.save('model.h5')



In [16]:
# Evaluar el modelo en datos de prueba y entrenamiento
evaluation_test = model.evaluate(X_test, Y_test)
print("Evaluación en datos de prueba:")
print("Pérdida:", evaluation_test[0])
print("Precisión:", evaluation_test[1])

evaluation_train = model.evaluate(X_train, Y_train)
print("Evaluación en datos de entrenamiento:")
print("Pérdida:", evaluation_train[0])
print("Precisión:", evaluation_train[1])

511/511 [==============================] - 51s 99ms/step - loss: 0.2883 - accuracy: 0.9343
Evaluación en datos de prueba:
Pérdida: 0.28826916217803955
Precisión: 0.9342837929725647
2043/2043 [==============================] - 103s 50ms/step - loss: 0.2753 - accuracy: 0.9365
Evaluación en datos de entrenamiento:
Pérdida: 0.27532312273979187
Precisión: 0.936543881893158


In [21]:
# Función para cargar, redimensionar y preprocesar una nueva imagen
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        resized_image = cv2.resize(image, (28, 28))
        preprocessed_image = resized_image.astype('float32') / 255.0
        return preprocessed_image
    else:
        return None

In [25]:


# Función para realizar una predicción de Pokémon a partir de una nueva imagen
def predict_pokemon(image_path):
    preprocessed_image = load_and_preprocess_image(image_path)
    if preprocessed_image is not None:
        preprocessed_image = preprocessed_image.reshape(1, 28, 28, 3)  # Agregar dimensión del lote (batch) a la imagen
        prediction_probabilities = model.predict(preprocessed_image)
        predicted_class_index = np.argmax(prediction_probabilities)
        predicted_pokemon = encoder.inverse_transform([predicted_class_index])[0]
        return predicted_pokemon
    else:
        return "No se pudo cargar la imagen."

# Ejemplo de cómo usar la función de predicción con una nueva imagen
new_image_path = '/Users/giovannyhidalgo/Desktop/b.png'
#new_image_path = '/Users/giovannyhidalgo/Desktop/archive/images/images1/pikachu.png'

predicted_pokemon = predict_pokemon(new_image_path)
print("Predicción de Pokémon:", predicted_pokemon)

1/1 [==============================] - 0s 41ms/step
Predicción de Pokémon: tapu-koko
